In [1]:
from collections import defaultdict
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px

In [7]:
battled_pairs = r'..\results\log_battle_arena_gpt4_as_judger.csv'
df = pd.read_csv(battled_pairs)
columns_to_inclusive = ['model_a', 'model_b', 'winner']
data = df[columns_to_inclusive]

data_no_nan = data[data['winner'].isna()==False]
data_no_ties = data[data['winner'].str.contains('tie', na=False) == False]
data_ties_only = data[data['winner'].str.contains('tie', na=False) == True]

In [14]:
def compute_elo(battles, K=4, SCALE=400, BASE=10, INIT_RATING=1000):
    rating = defaultdict(lambda: INIT_RATING)

    for rd, model_a, model_b, winner in battles[['model_a', 'model_b', 'winner']].itertuples():
        ra = rating[model_a]
        rb = rating[model_b]
        ea = 1 / (1 + BASE ** ((rb - ra) / SCALE))
        eb = 1 / (1 + BASE ** ((ra - rb) / SCALE))
        if winner == "model_a":
            sa = 1
        elif winner == "model_b":
            sa = 0
        elif winner == "tie" or winner == "tie(all bad)":
            sa = 0.5
        else:
            raise Exception(f"unexpected vote {winner}")
        rating[model_a] += K * (sa - ea)
        rating[model_b] += K * (1 - sa - eb)

    return rating

In [15]:
def preety_print_elo_ratings(ratings):
    df = pd.DataFrame([
        [n, elo_ratings[n]] for n in elo_ratings.keys()
    ], columns=["Model", "Elo rating"]).sort_values("Elo rating", ascending=False).reset_index(drop=True)
    df["Elo rating"] = (df["Elo rating"] + 0.5).astype(int)
    df.index = df.index + 1
    return df

elo_ratings = compute_elo(data_no_nan)
preety_print_elo_ratings(elo_ratings)

,Model,Elo rating
1,vicuna-13b,1151
2,koala-13b,1051
3,alpaca-13b,1004
4,stablelm-tuned-alpha-7b,983
5,chatglm-6b,969
6,llama-13b,955
7,dolly-v2-12b,952
8,oasst-pythia-12b,935
